In [1]:
from app.models import Session, Headline, Topic, Article
from app.analysis.topics import load_and_update_topics, score_headline

topics = load_and_update_topics()
topics

In [2]:
from sqlalchemy import or_
import pandas as pd
from functools import partial
topic = topics[0]
with Session() as s:
        headlines = s.query(Headline).join(Headline.article).filter(
            or_(*[Headline.title.like(f"%{word}%") for word in topic.essential]),
            Article.topic_id == None
        ).all()
        df = pd.DataFrame([(h.id, h.title) for h in headlines], columns=['id', 'title'])
        df['score'] = df['title'].apply(partial(score_headline, topic=topic))
df

In [3]:
df.describe()

In [4]:
df = df.sort_values(by='score', ascending=False)

In [5]:
df.sort_values(by='score', ascending=True)[df['score'] > 0.02]

In [7]:
import matplotlib.pyplot as plt

plt.hist(df[df['score'] > 0]['score'], bins=20)